# Example: Pause and Continue
This notebook shows how one can execute two functions for one experiment by setting the status to `paused`, and unpausing it later on with a different execution function. This example is heavily based on the `example_general_usage.ipynb` notebook.

To execute this notebook you need to install:
```
pip install py_experimenter
pip install scikit-learn
```

## Experiment Configuration File
First we define an experiment configuraiton file. Note that in comparison to the basic example two resultfields `paused_at_seconds` and `resumed_at_seconds` were added.

In [11]:
import os

content = """
PY_EXPERIMENTER:
  n_jobs: 1
  
  Database:
    provider: sqlite
    database: py_experimenter
    table: 
      name: example_pause_and_continue
      keyfields:
        dataset:
          type: VARCHAR(50)
          values: [iris]
        cross_validation_splits:
          type: INT
          values: [5]
        seed:
          type: INT
          values:
            start: 2
            stop: 7
            step: 2
        kernel:
          type: VARCHAR(50)
          values: [linear, poly, rbf, sigmoid]
      resultfields:
        pipeline: LONGTEXT
        train_f1: DECIMAL
        train_accuracy: DECIMAL
        test_f1: DECIMAL
        test_accuracy: DECIMAL
        paused_at_seconds: DOUBLE
        resumed_at_seconds: DOUBLE
      
  CUSTOM:
    path: sample_data

  CODECARBON:
    offline_mode: False
    measure_power_secs: 25
    tracking_mode: process
    log_level: error
    save_to_file: True
    output_dir: output/CodeCarbon
"""
# Create config directory if it does not exist
if not os.path.exists('config'):
    os.mkdir('config')
    
# Create config file
experiment_configuration_file_path = os.path.join('config', 'example_pause_and_continue.yml')
with open(experiment_configuration_file_path, "w") as f: 
  f.write(content)

## Defining Pausing Execution Function
Next we fill the table, define the execution function that gets paused after five seconds and run this execution function.

In [12]:
import datetime

from py_experimenter.experimenter import ExperimentStatus, PyExperimenter
from py_experimenter.result_processor import ResultProcessor

experimenter = PyExperimenter(experiment_configuration_file_path=experiment_configuration_file_path, name='example_notebook')

experimenter.fill_table_from_config()

def pause_after_5_seconds(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    import time
    time.sleep(5)
    result_processor.process_results({
        'paused_at_seconds': datetime.datetime.now().timestamp()
    })
    return ExperimentStatus.PAUSED



experimenter.execute(pause_after_5_seconds, max_experiments=1)

2024-02-27 09:23:56,918  | py-experimenter - INFO     | Found 4 keyfields
2024-02-27 09:23:56,920  | py-experimenter - WARNING  | No logtables given
2024-02-27 09:23:56,922  | py-experimenter - WARNING  | No custom section defined in config
2024-02-27 09:23:56,924  | py-experimenter - WARNING  | No codecarbon section defined in config
2024-02-27 09:23:56,926  | py-experimenter - INFO     | Initialized and connected to database
2024-02-27 09:23:56,975  | py-experimenter - INFO     | 12 rows successfully added to database. 0 rows were skipped.


[codecarbon INFO @ 09:23:57] [setup] RAM Tracking...
[codecarbon INFO @ 09:23:57] [setup] GPU Tracking...
[codecarbon INFO @ 09:23:57] No GPU found.
[codecarbon INFO @ 09:23:57] [setup] CPU Tracking...
[codecarbon WARNING @ 09:23:57] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 09:23:58] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1260P but we don't know it. Please contact us.
[codecarbon INFO @ 09:23:58] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 09:23:58] >>> Tracker's metadata:
[codecarbon INFO @ 09:23:58]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 09:23:58]   Python version: 3.9.0
[codecarbon INFO @ 09:23:58]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 09:23:58]   Available RAM : 15.475 GB
[codecarbon INFO @ 09:23:58]   CPU count: 16
[codecarbon INFO @ 09:23:58]   CPU model: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon 

## Showcase Paused Execution
Below we only show that the execution of the experiment with `id=1` has been paused.

In [13]:
experimenter.get_table()

,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,paused_at_seconds,resumed_at_seconds,end_date,error
0,1,iris,5,2,linear,2024-02-27 09:23:56,paused,2024-02-27 09:23:56,example_notebook,Worklaptop,None,None,None,None,None,1.709022e+09,None,2024-02-27 09:24:06,None
1,2,iris,5,4,linear,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
2,3,iris,5,6,linear,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
3,4,iris,5,2,poly,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
4,5,iris,5,4,poly,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
5,6,iris,5,6,poly,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
6,7,iris,5,2,rbf,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
7,8,iris,5,4,rbf,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
8,9,iris,5,6,rbf,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None
9,10,iris,5,2,sigmoid,2024-02-27 09:23:56,created,None,None,None,None,None,None,None,None,NaN,None,None,None


## Define resuming execution function

Lastly, we can define a second execution function that resumes the paused execution function. After running this execution function, the experiment is finished and the resulting table is shown.

In [14]:
import random

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

def resume_after_5_seconds(parameters: dict, result_processor: ResultProcessor, custom_config: dict):
    result_processor.process_results({
        'resumed_at_seconds': datetime.datetime.now().timestamp()
    })
    seed = parameters['seed']
    random.seed(seed)
    np.random.seed(seed)

    data = load_iris()
    # In case you want to load a file from a path
    # path = os.path.join(custom_config['path'], parameters['dataset'])
    # data = pd.read_csv(path)

    X = data.data
    y = data.target

    model = make_pipeline(StandardScaler(), SVC(kernel=parameters['kernel'], gamma='auto'))
    result_processor.process_results({
        'pipeline': str(model)
    })

    if parameters['dataset'] != 'iris':
        raise ValueError("Example error")

    scores = cross_validate(model, X, y,
                            cv=parameters['cross_validation_splits'],
                            scoring=('accuracy', 'f1_micro'),
                            return_train_score=True
                            )

    result_processor.process_results({
        'train_f1': np.mean(scores['train_f1_micro']),
        'train_accuracy': np.mean(scores['train_accuracy'])
    })

    result_processor.process_results({
        'test_f1': np.mean(scores['test_f1_micro']),
        'test_accuracy': np.mean(scores['test_accuracy'])
    })


experimenter.unpause_experiment(1, resume_after_5_seconds)

experimenter.get_table()

[codecarbon INFO @ 09:24:06] [setup] RAM Tracking...
[codecarbon INFO @ 09:24:06] [setup] GPU Tracking...
[codecarbon INFO @ 09:24:06] No GPU found.
[codecarbon INFO @ 09:24:06] [setup] CPU Tracking...
[codecarbon WARNING @ 09:24:06] No CPU tracking mode found. Falling back on CPU constant mode.


[codecarbon WARNING @ 09:24:07] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1260P but we don't know it. Please contact us.
[codecarbon INFO @ 09:24:07] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 09:24:07] >>> Tracker's metadata:
[codecarbon INFO @ 09:24:07]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 09:24:07]   Python version: 3.9.0
[codecarbon INFO @ 09:24:07]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 09:24:07]   Available RAM : 15.475 GB
[codecarbon INFO @ 09:24:07]   CPU count: 16
[codecarbon INFO @ 09:24:07]   CPU model: 12th Gen Intel(R) Core(TM) i7-1260P
[codecarbon INFO @ 09:24:07]   GPU count: None
[codecarbon INFO @ 09:24:07]   GPU model: None
[codecarbon INFO @ 09:24:10] Energy consumed for RAM : 0.000000 kWh. RAM Power : 5.803094387054443 W
[codecarbon INFO @ 09:24:10] Energy consumed for all CPUs : 0.000001 kWh. Total CPU Power : 42.5 W
[codecarbon INF

,ID,dataset,cross_validation_splits,seed,kernel,creation_date,status,start_date,name,machine,pipeline,train_f1,train_accuracy,test_f1,test_accuracy,paused_at_seconds,resumed_at_seconds,end_date,error
0,1,iris,5,2,linear,2024-02-27 09:23:56,done,2024-02-27 09:23:56,example_notebook,Worklaptop,"Pipeline(steps=[('standardscaler', StandardSca...",0.971667,0.971667,0.966667,0.966667,1.709022e+09,1.709022e+09,2024-02-27 09:24:10,None
1,2,iris,5,4,linear,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,3,iris,5,6,linear,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,4,iris,5,2,poly,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,5,iris,5,4,poly,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
5,6,iris,5,6,poly,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
6,7,iris,5,2,rbf,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
7,8,iris,5,4,rbf,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
8,9,iris,5,6,rbf,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None
9,10,iris,5,2,sigmoid,2024-02-27 09:23:56,created,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None


## CodeCarbon Entries
Note that for each execution a different `CodeCarbon` entry is created.

In [15]:
experimenter.get_codecarbon_table()

,ID,experiment_id,codecarbon_timestamp,project_name,run_id,duration_seconds,emissions_kg,emissions_rate_kg_sec,cpu_power_watt,gpu_power_watt,...,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,power_usage_efficiency,offline_mode
0,1,1,2024-02-27T09:24:06,codecarbon,74cb8e37-84ab-4e8f-9fd5-3e1e55d4b7c3,5.103890,2.475416e-05,0.000005,42.5,0.0,...,12th Gen Intel(R) Core(TM) i7-1260P,None,None,9.5312,52.4771,15.474918,machine,N,1.0,0
1,2,1,2024-02-27T09:24:10,codecarbon,cdf6e1c9-a168-4266-bb1b-eaa839c383e5,0.143559,4.489061e-07,0.000003,42.5,0.0,...,12th Gen Intel(R) Core(TM) i7-1260P,None,None,9.5312,52.4771,15.474918,machine,N,1.0,0
